<a href="https://colab.research.google.com/github/souvikjana1993/ISLR-python/blob/master/ULMFIT_DEMO_AV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing the necessary libraries

In [0]:
import fastai
from fastai.text import * 
import numpy as np
from functools import partial
import io
import os

In [2]:
#fetch the 20 news groups dataset
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

# Preprocessing Data

In [0]:
df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [0]:
#we convert the multilabels to binary labels where 1-10 is label 1 and 10-20 is label 2
df = df[df['label'].isin([1,10])]
df = df.reset_index(drop = True)

In [5]:
df['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

In [0]:
#Cleaning the data using regex commands. Retains only alphabets and removing else
df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

In [7]:
#removing stopwords
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

In [0]:
# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [0]:
#this part is not needed, for reference only
# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [12]:
df_trn.shape, df_val.shape

((710, 2), (474, 2))

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
learn = language_model_learner(data_lm, pretrained=URLs.WT103,arch=AWD_LSTM,drop_mult=0.7)

In [15]:
# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,6.714041,5.504577,0.241757,00:11


In [0]:
learn.save_encoder('ft_enc')

In [0]:
learn = text_classifier_learner(data_clas, arch=AWD_LSTM,drop_mult=0.7)
learn.load_encoder('ft_enc')

In [19]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
1,0.609922,0.339375,0.940928,00:35


In [20]:
# get predictions
preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,210,4
1,24,236
